In [1]:
import numpy as np
import random as rng
import numpy.random as numRand
import optproblems
from optproblems import cec2005
from optproblems import *
import typing 
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d

In [157]:
# mutation functions

# not actually used so I don't haev to comment it ;D
def bitFlip(genotype, flip_probability):
    p = flip_probability
    
    for i in range(len(genotype)):
        if p >= np.random.rand(1, 1):
            genotype[i] = not(genotype[i])
    return genotype
#end

# gaussian convolution
# Add gaussian-distributed random numbers to a random selection of values from the genotype
# variance: variance of the distribution random numbers will be generated from
# min: minimum number genotype's values can be 
# max: maximum number genotype's values can be
# noise_probability: probability that gaussian number will be added a value in the genotype
def gaussianConvolution(genotype, variance=1, min = -100, max = 100, noise_probability=0.5):
    p = noise_probability #probability of adding noise to a genotype value
    var = variance        #variance of the gaussian distribution which will be added to a genotype val
    mu = 0                #mean of the normal distribution
    
    for i in range(len(genotype)): #for every genotype member
        # if the noise probability is greater than a random number (between 0 and 1)
        if p >= np.random.rand(1, 1): #a 1x1 array of random numbers (a.k.a a random number)
            #generate a random nnumber from a gaussian distribution
            randy_num = rng.normalvariate(mu, var)
            
            #until (current genotype value + randy_num) is between min and max
            while not (min <= genotype[i] + randy_num <= max):
                randy_num = rng.normalvariate(mu, var)
            #end
            
            genotype[i] = (genotype[i] + randy_num) #update current value from genotype
        #end
    return genotype
#end

mutate_functions = {"bit flip":bitFlip, "g convolve":gaussianConvolution}

In [153]:
# test mutation

pop = np.array([2, 2, 2, 2, 1, 1, 1, 1], dtype="float")
mut_pop = mutate_functions["g convolve"](pop, 1, 0.5)
mut_pop

array([2.53358443, 1.79421467, 2.51870221, 0.69723394, 1.33371562,
       3.2462227 , 1.80373636, 0.88083599])

In [342]:
# recombination

# swap values between 2 vectors starting from a random index and ending at another random index
# (index circularly)
# vec_1: a vector
# vec_2: a vector
def twoPtCrossover(vec_1, vec_2):
    #TODO - implement
    vec_3, vec_4 = vec_1, vec_2
    return vec_3, vec_4
#end

#swap values between 2 vectors starting from a random index
#vec_1: a vector
#vec_2: a vector
def onePtCrossover(vec_1, vec_2):
    #TODO handle unequal-length vecs
    #TODO handle empty vecs
    vec_1 = vec_1.copy()
    vec_2 = vec_2.copy()
    
    # choose a random integer (between bidx and pop length) and
    # swap vector values between that index and the vectors' ends
    cross_beg_idx = int(rng.uniform(0, len(vec_1)))
    
    # if crossover start is greater than first index 
    # (crossing from zero would just output the same 2 vectors in reverse order)
    if cross_beg_idx > 0:
        # swap the values
        for idx in range(cross_beg_idx, len(vec_1)):
            swap = vec_1[idx] #store the value to be swapped
            vec_1[idx] = vec_2[idx]
            vec_2[idx] = swap #swap the value to be swapped
        #end
    #end
    return vec_1, vec_2
#end

recombine_fns = {"1-pt-crossover":onePtCrossover}

In [5]:
# test recombination
pop_1 = [4.0, 3.2, 1.0, 0.5, 6.2]
pop_2 = [10.0, 15.0, 12.2, 9.5, 5.0]

pop_3, pop_4 = onePtCrossover(pop_1, pop_2)

print(pop_3)
pop_4

[4.0, 3.2, 1.0, 9.5, 5.0]


[10.0, 15.0, 12.2, 0.5, 6.2]

In [343]:
#selection

# Assess a genotype's fitness score in relation to a given problem function
# In GA, allows GA to find genotypes with greatest fitness and "naturally select" them
# genotype: a potential solution
# problem: the solution will be assessed on the output it receives when applied to this problem
def assessFitness(genotype, problem):
    if genotype == None:
        fitness = np.inf
    else:
        solution = Individual(genotype)
        problem.evaluate(solution)
        fitness = solution.objective_values
    #end
    
    return fitness
#end

# select the best genotype from the population according to a tournament process
# population: to select individuals from
# tournament_size: size of the group considered for selection
# problem: population members' fitness will be assessed according to their score on this problem
def tournamentSelect(population, tournament_size, problem, debug = False):
    t = tournament_size
    
    select_idx = int(rng.uniform(0, len(population)))
    best_idx = select_idx
    best = population[select_idx]
    
    # for every population element between second and "tournament size"...
    for i in range(0, t):
        # choose a "next" at random from pop
        select_idx = int(rng.uniform(0, len(population)))
        next = population[select_idx]
        if debug:
            print("Fighter: {}\t Fitness: {}".format(next, assessFitness(next, problem)))
        #end
        
        # if fitness of Next better than best...
        if assessFitness(next, problem) < assessFitness(best, problem):
            best = next
        #end
    #end
    
    return best
#end

In [378]:
# test assessFitness()

# print(assessFitness([-19.9, 50.35], problem = optproblems.Problem(cec2005.F1(2))))
print(assessFitness([3.769, -0.896], problem = optproblems.Problem(cec2005.F1(2))))
print(assessFitness([2, -2], problem = optproblems.Problem(cec2005.F1(2))))

4981.51360162
4965.47090162


In [317]:
# test tournamentSelect()

population = [[-15, 13.9], [-13, 12], [-19.9, 50.35], [5, 50], [-15, 0], [-13, 42], [-3, 59], [4, -4], 
              [-13, 13], [15, 15], [-15, 13.9], [-19.9, 50.35], [-15, 13], [-15, 13], [-10, 0], 
              [-15, 13.9], [8, 2], [-19.9, 90.35], [6, 4], [-19.9, 10.35]]

king_of_fighters = tournamentSelect(population, 10, problem = optproblems.Problem(cec2005.F1(2)), debug =True)

print("King: {}".format(king_of_fighters))

Fighter: [-10, 0]	 Fitness: 3878.38570162
Fighter: [-13, 13]	 Fitness: 2349.11690162
Fighter: [-19.9, 90.35]	 Fitness: 915.9306516199995
Fighter: [-15, 13.9]	 Fitness: 2166.0594816200005
Fighter: [-19.9, 50.35]	 Fitness: -0.07734837999987576
Fighter: [-3, 59]	 Fitness: 868.5641016200002
Fighter: [-13, 13]	 Fitness: 2349.11690162
Fighter: [-13, 12]	 Fitness: 2441.91670162
Fighter: [5, 50]	 Fitness: 1592.7527016200002
Fighter: [4, -4]	 Fitness: 5382.318101620001
King: [-19.9, 50.35]


In [8]:
#copy eerything except a given element from a list
def copyExcept(copy_this, dont_copy_this):
    list_clone = list()
    for i in range(0, len(copy_this)):
        if copy_this[i] != dont_copy_this:
            list_clone.append(copy_this[i])
    #end
    
    return list_clone
#end

In [17]:
dont_copy = [3, 3]
test_pop = [[1, 1], [2, 2], dont_copy]
print(test_pop)

clone = copyExcept(test_pop, dont_copy)
print(clone)

[[1, 1], [2, 2], [3, 3]]
[[1, 1], [2, 2]]


In [384]:
# genetic algorithm
PROBLEM_SIZE = 2
fit_hist = list()

#TODO docs
# generation_limit:
# population_size:
# problem_size:
# problem:
def geneticAlgorithm(generation_limit = 500, population_size = 100, problem_size = PROBLEM_SIZE, problem = optproblems.Problem(cec2005.F1(PROBLEM_SIZE))):
    # TODO optimise for fitness checks
    # create population
    pop = list()
    for i in range(population_size):
        pop.append(list(numRand.rand(1, problem_size)[0]))
    #end
    
    glob_best = None
    glob_best_fit = np.inf
    generation = 0
    
    # assess fitnesses and assign best until best is the ideal solution
    while generation < generation_limit:
#         int_pop = [[int(g[0]), int(g[1])] for g in pop]
#         print("Pop: \t\t{}".format(int_pop))
        
        generation += 1
        best = None
        best_fit = np.inf
        fit_list = list()
        
        # find the most selectablw genotype in the generation
        for idx in range(0, population_size):
            fit_list.append(assessFitness(pop[idx], problem))
            if  fit_list[idx] < best_fit:
#                 print("New best: {}".format(pop[idx]))
#                 print("{} -> {} = {}".format(best_fit, int(assessFitness(pop[idx], problem)), int(fit_list[idx])))
                best = pop[idx].copy()
                best_fit = fit_list[idx].copy()
            #end
        #end
#         print("New gen")
        
        fit_hist.append(best_fit.copy())
        
        if  best_fit < glob_best_fit:
            glob_best = best.copy()
            glob_best_fit = best_fit.copy()
        #end
        
        # create next generation (Q in the pseudo code)
        next_gen_pop = list()
        for i in range(int(population_size/2)):
            # select parents
            parent_1 = tournamentSelect(pop.copy(), int(len(pop)/2), problem)
            copy_pop = copyExcept(pop.copy(), parent_1)
            parent_2 = tournamentSelect(copy_pop, int(len(pop)/2), problem)
            
            # birth some babbies using crossover
            child_1, child_2 = onePtCrossover(parent_1, parent_2)
            
            next_gen_pop.append(mutate_functions["g convolve"](child_1))
            next_gen_pop.append(mutate_functions["g convolve"](child_2))
        #end
        
#         int_pop = [[int(g[0]), int(g[1])] for g in pop]
#         next_ints = [[int(g[0]), int(g[1])] for g in next_gen_pop]
#         int_fits = [int(f) for f in fit_list]
#         print("Pop: \t\t{}".format(int_pop))
#         print("Next pop: \t{}".format(next_ints))
#         print("Fits: {}".format(int_fits))
#         print(assessFitness(best, problem))
        print("Best : {}\nBest fit : {}\nGen {}/{}".format(best, best_fit, generation, generation_limit))
        
        pop = next_gen_pop.copy()
    #end
    
    return glob_best_fit
#end



In [385]:
fitness = geneticAlgorithm(generation_limit = 500, population_size = 10, problem_size = PROBLEM_SIZE, problem = optproblems.Problem(cec2005.F1(PROBLEM_SIZE)))

print("\nFitness: {}".format(fitness))

Best : [0.2004442814541617, 0.9661510879208148]
Best fit : 4467.544613624
Gen 1/500
Best : [-0.321736563919347, 2.2701039972825146]
Best fit : 4277.166640081678
Gen 2/500
Best : [-0.9443493888542872, 2.2701039972825146]
Best fit : 4229.002735208226
Gen 3/500
Best : [-1.7449470579545798, 2.2701039972825146]
Best fit : 4168.209748659253
Gen 4/500
Best : [-1.2188127013796248, 3.2928881908985073]
Best fit : 4093.223062277866
Gen 5/500
Best : [-2.528120662789913, 4.825118003526355]
Best fit : 3827.128470294714
Gen 6/500
Best : [-2.528120662789913, 4.760896662583017]
Best fit : 3834.0781046974116
Gen 7/500
Best : [-3.181384888810562, 6.388560850955237]
Best fit : 3612.85486142589
Gen 8/500
Best : [-2.528120662789913, 6.9600981098935195]
Best fit : 3600.7894427120727
Gen 9/500
Best : [-2.977981650569029, 8.097252480645158]
Best fit : 3451.062617598928
Gen 10/500
Best : [-2.528120662789913, 9.825875476648537]
Best fit : 3311.306305247065
Gen 11/500
Best : [-4.905768753247794, 9.825875476648537

Gen 116/500
Best : [-39.09068232012486, 58.90735114521988]
Best fit : -449.95100721854556
Gen 117/500
Best : [-39.09068232012486, 58.90735114521988]
Best fit : -449.95100721854556
Gen 118/500
Best : [-39.13344719905016, 58.90735114521988]
Best fit : -449.9680990782681
Gen 119/500
Best : [-39.13344719905016, 58.90735114521988]
Best fit : -449.9680990782681
Gen 120/500
Best : [-39.13344719905016, 58.90735114521988]
Best fit : -449.9680990782681
Gen 121/500
Best : [-39.13344719905016, 58.90735114521988]
Best fit : -449.9680990782681
Gen 122/500
Best : [-39.13344719905016, 58.90735114521988]
Best fit : -449.9680990782681
Gen 123/500
Best : [-39.13344719905016, 58.871678513449154]
Best fit : -449.96735814553
Gen 124/500
Best : [-39.13344719905016, 58.871678513449154]
Best fit : -449.96735814553
Gen 125/500
Best : [-39.13344719905016, 58.871678513449154]
Best fit : -449.96735814553
Gen 126/500
Best : [-39.13344719905016, 58.871678513449154]
Best fit : -449.96735814553
Gen 127/500
Best : [-39

Gen 219/500
Best : [-39.32986357470686, 58.970329797658664]
Best fit : -449.9947169535855
Gen 220/500
Best : [-39.32986357470686, 58.970329797658664]
Best fit : -449.9947169535855
Gen 221/500
Best : [-39.32986357470686, 58.970329797658664]
Best fit : -449.9947169535855
Gen 222/500
Best : [-39.32986357470686, 58.970329797658664]
Best fit : -449.9947169535855
Gen 223/500
Best : [-39.32986357470686, 58.970329797658664]
Best fit : -449.9947169535855
Gen 224/500
Best : [-39.32986357470686, 58.970329797658664]
Best fit : -449.9947169535855
Gen 225/500
Best : [-39.32986357470686, 58.970329797658664]
Best fit : -449.9947169535855
Gen 226/500
Best : [-39.34125031598524, 58.970329797658664]
Best fit : -449.9941782025533
Gen 227/500
Best : [-39.34125031598524, 58.970329797658664]
Best fit : -449.9941782025533
Gen 228/500
Best : [-39.34125031598524, 58.970329797658664]
Best fit : -449.9941782025533
Gen 229/500
Best : [-39.27602981159495, 58.970329797658664]
Best fit : -449.99375297318556
Gen 230/5

Gen 318/500
Best : [-39.209358909027245, 58.765888773308085]
Best fit : -449.9715263157826
Gen 319/500
Best : [-39.209358909027245, 58.765888773308085]
Best fit : -449.9715263157826
Gen 320/500
Best : [-39.43974676514781, 58.81776398086719]
Best fit : -449.9769088790023
Gen 321/500
Best : [-39.43974676514781, 58.87892971165902]
Best fit : -449.98321545164816
Gen 322/500
Best : [-39.209358909027245, 58.81776398086719]
Best fit : -449.9827389990231
Gen 323/500
Best : [-39.73604121624196, 58.81776398086719]
Best fit : -449.8133579030458
Gen 324/500
Best : [-39.56627159703886, 58.87958886382897]
Best fit : -449.93488254836734
Gen 325/500
Best : [-39.56627159703886, 58.87958886382897]
Best fit : -449.93488254836734
Gen 326/500
Best : [-38.94564811048816, 59.100658678850486]
Best fit : -449.8255555062952
Gen 327/500
Best : [-39.16012185394107, 58.81776398086719]
Best fit : -449.97021706873994
Gen 328/500
Best : [-38.94564811048816, 59.100658678850486]
Best fit : -449.8255555062952
Gen 329/50

Gen 417/500
Best : [-39.36343324256209, 58.74961548832466]
Best fit : -449.97475889046154
Gen 418/500
Best : [-39.400269046812625, 59.076142024284174]
Best fit : -449.96112966044166
Gen 419/500
Best : [-39.584347734138525, 58.75937636242869]
Best fit : -449.9060253394465
Gen 420/500
Best : [-39.54238751114707, 58.75937636242869]
Best fit : -449.92712861448894
Gen 421/500
Best : [-39.32224921735021, 58.82955787130259]
Best fit : -449.9949448786306
Gen 422/500
Best : [-39.54238751114707, 58.82955787130259]
Best fit : -449.94192749213556
Gen 423/500
Best : [-39.54238751114707, 58.82955787130259]
Best fit : -449.94192749213556
Gen 424/500
Best : [-39.469017694869954, 58.82955787130259]
Best fit : -449.97036601488907
Gen 425/500
Best : [-39.32245586788798, 58.41649161599765]
Best fit : -449.76620490792936
Gen 426/500
Best : [-39.32245586788798, 59.3557243762306]
Best fit : -449.79211271168714
Gen 427/500
Best : [-39.64183180001255, 59.02790640703305]
Best fit : -449.874759367099
Gen 428/500